In [1]:
import tensorflow as tf
import numpy as np
from mpcm import MPCM

In [2]:
flags = tf.app.flags
flags.DEFINE_integer('train_epoch', 100, 'Training epoch')
flags.DEFINE_integer('test_epoch', 1, 'Test for every n training epoch')
flags.DEFINE_integer("batch_size", 32, "Size of batch (32)")
flags.DEFINE_integer("dim_perspective", 20, "Maximum number of perspective (20)")
flags.DEFINE_integer("dim_embed_word", 300, "Dimension of word embedding (300)")
flags.DEFINE_integer("dim_rnn_cell", 100, "Dimension of RNN cell (100)")
flags.DEFINE_integer("dim_hidden", 100, "Dimension of hidden layer")
flags.DEFINE_integer("num_paraphrase", 1, "Maximum number of question paraphrasing")
flags.DEFINE_integer("rnn_layer", 1, "Layer number of RNN ")
flags.DEFINE_integer("context_maxlen", 0, "Predefined context max length")
flags.DEFINE_integer("validation_cnt", 100, "Number of model validation")
flags.DEFINE_float("rnn_dropout", 0.5, "Dropout of RNN cell")
flags.DEFINE_float("hidden_dropout", 0.5, "Dropout rate of hidden layer")
flags.DEFINE_float("embed_dropout", 0.8, "Dropout rate of embedding layer")
flags.DEFINE_float("learning_rate", 0.00162, "Initial learning rate of the optimzier")
flags.DEFINE_float("decay_rate", 1.00, "Decay rate of learning rate (0.99)")
flags.DEFINE_float("decay_step", 100, "Decay step of learning rate")
flags.DEFINE_float("max_grad_norm", 5.0, "Maximum gradient to clip")
flags.DEFINE_boolean("embed_trainable", False, "True to optimize embedded words")
flags.DEFINE_boolean("test", False, "True to run only iteration 5")
flags.DEFINE_boolean("debug", False, "True to show debug message")
flags.DEFINE_boolean("save", False, "True to save model after testing")
flags.DEFINE_boolean("sample_params", False, "True to sample parameters")
flags.DEFINE_string("model", "m", "b: basic, m: mpcm, q: ql_mpcm")
flags.DEFINE_string('train_path', './data/train-v1.1.json', 'Training dataset path')
flags.DEFINE_string('dev_path', './data/dev-v1.1.json',  'Development dataset path')
flags.DEFINE_string('pred_path', './result/dev-v1.1-pred.json', 'Prediction output path')
flags.DEFINE_string('glove_path', \
        '~/common/glove/glove.6B.'+ str(tf.app.flags.FLAGS.dim_embed_word) +'d.txt', 'embed path')
flags.DEFINE_string('validation_path', './result/validation.txt', 'Validation file path')
flags.DEFINE_string('checkpoint_dir', './result/ckpt/', 'Checkpoint directory')
FLAGS = flags.FLAGS

In [4]:
import sys
import pprint
import argparse
import datetime
import random
import copy

from model import Basic
from ql_mpcm import QL_MPCM
from time import gmtime, strftime
from dataset import read_data, build_dict, load_glove, preprocess
from run import train, test

pretrained_glove, word2idx
def main(_):
    # Load dataset once
    expected_version = '1.1'
    train_path = FLAGS.train_path
    dev_path = FLAGS.dev_path
    train_dataset = read_data(train_path, expected_version)
    dev_dataset = read_data(dev_path, expected_version)

    # Preprocess dataset
    saved_params = FLAGS.__flags
    word2idx, idx2word, c_maxlen, q_maxlen = build_dict(train_dataset, saved_params)
    pretrained_glove, word2idx, idx2word = load_glove(word2idx, saved_params)
    if saved_params['context_maxlen'] > 0: 
        c_maxlen = saved_params['context_maxlen']

    train_dataset = preprocess(train_dataset, word2idx, c_maxlen, q_maxlen)
    dev_dataset = preprocess(dev_dataset, word2idx, c_maxlen, q_maxlen)
    saved_params['context_maxlen'] = c_maxlen
    saved_params['question_maxlen'] = q_maxlen
    saved_params['voca_size'] = len(word2idx)
    saved_params['dim_output'] = c_maxlen

if __name__ == "__main__":
    tf.app.run()

Top 20 frequent words among 103027
[('of', 130668), ('in', 90564), ('and', 86029), ('to', 70981), ('what', 51635), ('is', 37323), ('was', 36275), ('as', 26334), ('for', 25093), ('by', 21712), ('that', 20885), ('with', 20399), ('on', 19459), ('did', 16969), ('are', 16715), ('from', 16562), ('which', 15477), ('were', 13661), ('who', 12626), ('or', 11336)]
229–280 1
digit cnt 4517
alpha cnt 98643
Dictionary size 103027
[('', 0), ('0', 1), ('00', 2), ('000', 3), ('0000', 4), ('0000222556', 5), ('0001', 6), ('00018', 7), ('0002', 8), ('00028', 9), ('0003', 10), ('00042', 11), ('00043', 12), ('0005', 13), ('00054', 14), ('00071', 15), ('00079', 16), ('000–20', 17), ('001', 18), ('0012', 19)]
Maxlen of C:581, Q:34, A:35
Glove Loading...
Glove Loading Done 0:00:43.227872 400000
apple: 5506 [-0.20842, -0.019668, 0.063981, -0.71403, -0.21181]
Pretrained vectors (67223, 300) unknown 35806
Dictionary Change 103027 to 67223 67223
Passage: 18896, Question: 87599
Passage: 2067, Question: 10570


SystemExit: 

/home/jinhyuk/py3venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
params = copy.deepcopy(FLAGS.__flags)
pprint.PrettyPrinter().pprint(params)

# Make model and run experiment
if params['model'] == 'm':
    my_model = MPCM(params, initializer=[pretrained_glove, word2idx])
elif params['model'] == 'q':
    my_model = QL_MPCM(params, initializer=[pretrained_glove, word2idx])
elif params['model'] == 'b':
    my_model = Basic(params, initializer=[pretrained_glove, word2idx])
else:
    assert False, "Check your version %s" % params['model']

my_model.load(params['checkpoint_dir'])

{'batch_size': 32,
 'checkpoint_dir': './result/ckpt/',
 'context_maxlen': 581,
 'debug': False,
 'decay_rate': 1.0,
 'decay_step': 100,
 'dev_path': './data/dev-v1.1.json',
 'dim_embed_word': 300,
 'dim_hidden': 100,
 'dim_output': 581,
 'dim_perspective': 20,
 'dim_rnn_cell': 100,
 'embed_dropout': 0.8,
 'embed_trainable': False,
 'glove_path': '~/common/glove/glove.6B.300d.txt',
 'hidden_dropout': 0.5,
 'learning_rate': 0.00162,
 'max_grad_norm': 5.0,
 'model': 'm',
 'num_paraphrase': 1,
 'pred_path': './result/dev-v1.1-pred.json',
 'question_maxlen': 34,
 'rnn_dropout': 0.5,
 'rnn_layer': 1,
 'sample_params': False,
 'save': False,
 'test': False,
 'test_epoch': 1,
 'train_epoch': 100,
 'train_path': './data/train-v1.1.json',
 'validation_cnt': 100,
 'validation_path': './result/validation.txt',
 'voca_size': 67223}


NameError: name 'pretrained_glove' is not defined